# Carga y Preprocesamiento de Datos
* Carga los datos del archivo CSV utilizando NumPy.
* Realiza un preprocesamiento básico para asegurarte de que los datos estén limpios y listos para su análisis.



In [25]:
import numpy as np
def cargar_datos(ruta_archivo):
    datos = np.genfromtxt(ruta_archivo, delimiter=',', skip_header=1, dtype=str)
    return datos

# 1. Filtrar por producto específico
def filtrar_por_producto(datos, producto_filtro):
    # Filtra los datos donde el producto es igual al producto_filtro
    return datos[datos[:, 1] == producto_filtro]

# 2. Filtrar por tienda específica
def filtrar_por_tienda(datos, tienda_filtro):
    # Filtra los datos donde la tienda es igual a tienda_filtro
    return datos[datos[:, 3] == tienda_filtro]

# Función para calcular el total de ventas por producto y tienda (ya existente)
def total_ventas_por_producto_tienda(datos):
    productos = datos[:, 1]  # Columna de productos
    tiendas = datos[:, 3]  # Columna de tiendas
    ingresos = datos[:, 2].astype(float)  # Columna de ingresos

    # Obtener productos y tiendas únicos
    productos_unicos, idx_productos = np.unique(productos, return_inverse=True)
    tiendas_unicas, idx_tiendas = np.unique(tiendas, return_inverse=True)

    # Crear una matriz para almacenar las sumas de ingresos por producto y tienda
    total_ventas = np.zeros((len(productos_unicos), len(tiendas_unicas)))

    # Sumar los ingresos por producto y tienda
    for i in range(len(ingresos)):
        total_ventas[idx_productos[i], idx_tiendas[i]] += ingresos[i]

    return productos_unicos, tiendas_unicas, total_ventas

# Función para calcular el promedio de ventas diarias por producto y tienda (ya existente)
def promedio_ventas_por_producto_tienda(datos):
    productos = datos[:, 1]
    tiendas = datos[:, 3]
    fechas = datos[:, 0]
    ingresos = datos[:, 2].astype(float)

    # Obtener productos y tiendas únicos
    productos_unicos, idx_productos = np.unique(productos, return_inverse=True)
    tiendas_unicas, idx_tiendas = np.unique(tiendas, return_inverse=True)

    # Crear una matriz para almacenar los ingresos por día y tienda
    suma_ingresos = np.zeros((len(productos_unicos), len(tiendas_unicas)))
    conteo_dias = np.zeros((len(productos_unicos), len(tiendas_unicas)))

    # Usamos un array de tipo object para almacenar fechas
    fechas_unicas = np.empty_like(suma_ingresos, dtype=object)  # Cambiamos a dtype=object porque no acepta string

    # Sumar ingresos y contar días únicos
    for i in range(len(ingresos)):
        prod_idx = idx_productos[i]
        tienda_idx = idx_tiendas[i]

        # Evitar contar la misma fecha más de una vez
        fecha_actual = fechas[i]
        if fechas_unicas[prod_idx, tienda_idx] != fecha_actual:
            conteo_dias[prod_idx, tienda_idx] += 1
            fechas_unicas[prod_idx, tienda_idx] = fecha_actual

        suma_ingresos[prod_idx, tienda_idx] += ingresos[i]

    # Calcular el promedio dividiendo el total de ventas entre los días
    promedio_ventas = np.divide(suma_ingresos, conteo_dias, out=np.zeros_like(suma_ingresos), where=conteo_dias!=0)

    return productos_unicos, tiendas_unicas, promedio_ventas

# Función para identificar mayores y menores ventas (ya existente)
def identificar_mayores_menores_ventas(productos_unicos, tiendas_unicas, total_ventas):
    # Encontrar la venta máxima y mínima
    max_ventas = np.max(total_ventas)
    min_ventas = np.min(total_ventas)

    # Encontrar los índices de los valores máximos y mínimos
    max_indices = np.where(total_ventas == max_ventas)
    min_indices = np.where(total_ventas == min_ventas)

    # Obtener los productos y tiendas correspondientes
    productos_max = productos_unicos[max_indices[0]]
    tiendas_max = tiendas_unicas[max_indices[1]]

    productos_min = productos_unicos[min_indices[0]]
    tiendas_min = tiendas_unicas[min_indices[1]]

    return (productos_max, tiendas_max, max_ventas), (productos_min, tiendas_min, min_ventas)

# 3. Operaciones adicionales: suma, resta, multiplicación y división para estadísticas
def operaciones_adicionales(datos):
    ingresos = datos[:, 2].astype(float)  # Columna de ingresos

    # Operaciones básicas
    suma_total = np.sum(ingresos)
    resta_total = np.sum(ingresos - np.mean(ingresos))  # Restar el promedio a cada valor
    multiplicacion_total = np.prod(ingresos)  # Producto de todos los ingresos
    division_promedio = np.divide(ingresos, np.mean(ingresos))  # Dividir cada ingreso por el promedio

    return suma_total, resta_total, multiplicacion_total, division_promedio

if __name__ == "__main__":

    ruta_archivo = '/content/Datos_tienda.csv'
    datos = cargar_datos(ruta_archivo)

    # Filtrar datos para un producto específico
    producto_filtro = '19'  # Cambia por el nombre del producto para el filtro deseado
    datos_producto = filtrar_por_producto(datos, producto_filtro)
    print(f"Datos filtrados para el producto '{producto_filtro}':")
    print(datos_producto)

    # Filtrar datos para una tienda específica
    tienda_filtro = '1'  # Cambia por el nombre de la tienda para el filtro deseado
    datos_tienda = filtrar_por_tienda(datos, tienda_filtro)
    print(f"\nDatos filtrados para la tienda '{tienda_filtro}':")
    print(datos_tienda)

    # Total de ventas por producto y tienda
    productos, tiendas, total_ventas = total_ventas_por_producto_tienda(datos)
    print("\nTotal de ventas por producto y tienda:")
    print(total_ventas)

    # Promedio de ventas diarias por producto y tienda
    productos, tiendas, promedio_ventas = promedio_ventas_por_producto_tienda(datos)
    print("\nPromedio de ventas diarias por producto y tienda:")
    print(promedio_ventas)

    # Identificar mayores y menores ventas
    (productos_max, tiendas_max, max_ventas), (productos_min, tiendas_min, min_ventas) = identificar_mayores_menores_ventas(productos, tiendas, total_ventas)
    print(f"\nMayor venta: Producto(s) {productos_max} en Tienda(s) {tiendas_max} con {max_ventas}")
    print(f"Menor venta: Producto(s) {productos_min} en Tienda(s) {tiendas_min} con {min_ventas}")

    # Operaciones adicionales
    suma, resta, multiplicacion, division = operaciones_adicionales(datos)
    print(f"\nSuma total de ingresos: {suma}")
    print(f"Resta total de ingresos (desviación del promedio): {resta}")
    print(f"Multiplicación total de ingresos: {multiplicacion}")
    print(f"División de ingresos sobre el promedio: {division}")

Datos filtrados para el producto '19':
[['01/01/2019' '19' '8936' '1']
 ['15/02/2019' '19' '5931' '1']
 ['13/04/2019' '19' '5079' '1']
 ['19/05/2019' '19' '8496' '1']
 ['30/06/2019' '19' '8973' '1']
 ['10/11/2019' '19' '7491' '1']
 ['26/11/2019' '19' '6074' '1']
 ['25/02/2020' '19' '5901' '1']
 ['10/04/2020' '19' '5963' '1']
 ['10/06/2020' '19' '8460' '1']
 ['04/07/2020' '19' '6515' '1']
 ['08/07/2020' '19' '7305' '1']
 ['22/08/2020' '19' '7974' '1']
 ['08/09/2020' '19' '6158' '1']
 ['23/01/2021' '19' '9670' '1']
 ['06/03/2021' '19' '6964' '1']
 ['16/04/2021' '19' '9442' '1']
 ['24/04/2021' '19' '6909' '1']
 ['13/05/2021' '19' '9954' '1']
 ['21/05/2021' '19' '8735' '1']
 ['04/07/2021' '19' '5092' '1']
 ['08/08/2021' '19' '8119' '1']
 ['17/08/2021' '19' '7918' '1']
 ['24/11/2021' '19' '9645' '1']
 ['10/01/2019' '19' '9378' '2']
 ['26/04/2019' '19' '8954' '2']
 ['18/05/2019' '19' '8925' '2']
 ['29/06/2019' '19' '7741' '2']
 ['09/07/2019' '19' '5102' '2']
 ['11/08/2019' '19' '9830' '2']
 